#### Load warped ROI masks and only re-create NEUROPIL mask

In [ ]:
maskinfo = get_mask_info(TID, RID, nslices=nslices, rootdir=rootdir)
maskdict_path = os.path.join(TID['DST'], 'MASKS.hdf5')

In [ ]:
# mask_write_path = os.path.join(TID['DST'], 'MASKS.hdf5') -- this is 'maskdict_path'

maskdict_path = get_masks(maskdict_path, maskinfo, RID,
                                  save_warp_images=False,
                                  do_neuropil_correction=True,
                                  niter=niter,
                                  rootdir=rootdir)

In [ ]:

# 'mask_arr' is output of masks_to_normed_array(masks_aligned) 
# --- original shape of masks_aligned is:  Assumes masks.shape = (d1, d2, nrois)
# --- normalizes and reshapes to : np.reshape(masks[:,:,r], (d,), order='C'), where d=d1*d2

# 'mask_arr' is saved in h5:   filegrp[curr_slice]['maskarray']
# 'np_maskarray' is saved in h5:   filegrp[curr_slice]['np_maskarray']

# MASKS = h5py.File(mask_write_path, 'w')
# MASKS.attrs['source_file'] = maskinfo['filepath']
# MASKS.attrs['roi_id'] = str(RID['roi_id'])
# MASKS.attrs['rid_hash'] = str(RID['rid_hash'])
# MASKS.attrs['roi_type'] = str(RID['roi_type'])

# try:
#     maskfile = h5py.File(str(maskinfo['filepath']), 'r')
#     for fidx, curr_file in enumerate(maskinfo['filenames']):
#         if curr_file not in MASKS.keys():
#             filegrp = MASKS.create_group(curr_file)
#         else:
#             filegrp = MASKS[curr_file]
#         # DO THE SHIT
#
# except Exception as e:
#     print "------------------------------------------"
#     print "*** ERROR creating masks: %s, %s ***" % (curr_file, curr_slice)
#     traceback.print_exc()
#     print "------------------------------------------"

# finally:
#     MASKS.close()
#     maskfile.close()


# Check if should create neuropil masks:
if do_neuropil_correction:
    np_masks = create_neuropil_masks(masks_aligned, niterations=niter)
    npil_arr = masks_to_normed_array(np_masks)
    npil = filegrp.create_dataset('/'.join([curr_slice, 'np_maskarray']), npil_arr.shape, npil_arr.dtype)
    npil[...] = npil_arr
    npil.attrs['niterations'] = niter

#### Apply new NP mask to extract traces

In [ ]:
filetraces_dir = os.path.join(TID['DST'], 'files')

In [ ]:
print "--- Using SUBTRACTION method, (global) correction-factor: ", np_correction_factor
filetraces_dir = append_neuropil_subtraction(maskdict_path,
                                             np_correction_factor,
                                             filetraces_dir,
                                             create_new=create_new,
                                             rootdir=rootdir)